In [1]:
from torch.utils.data import DataLoader
from tqdm import tqdm

%reload_ext autoreload
%autoreload 2

from src import *

In [2]:
dataset = data.TinyStoriesDataset(1024)

Tokenizing Stories: 100%|██████████| 50000/50000 [00:24<00:00, 2056.60 stories/s]


In [3]:
# Sanity checking dataloader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, drop_last=True)
for i, (data, label) in enumerate(dataloader):
    assert data.shape == (4, 1024)
    assert label.shape == (4, 1024)
    assert torch.equal(data[:, 1:], label[:, :-1])
print("No issues found")

No issues found
